# Regression Week 2: Multiple Regression (gradient descent)

In the first notebook we explored multiple regression using graphlab create. Now we will use graphlab along with numpy to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent. You will:
* Add a constant column of 1's to a graphlab SFrame to account for the intercept
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [1]:
train <- read.csv('kc_house_train_data.csv', header = TRUE)
test <- read.csv('kc_house_test_data.csv', header = TRUE)
sales <- rbind(train,test)

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the other Week 2 notebook. For this notebook, however, we will work with the existing features.

# Convert to Matrix Vector

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

With this in mind, complete the following function (where there's an empty line you should write a line of code that does what the comment above indicates)

**Please note you will need GraphLab Create version at least 1.7.1 in order for .to_numpy() to work!**

In [2]:
get_matrix_data <- function(data_dframe, features, output){
    data_dframe$constant = 1
    features = c('constant',features)
    feature_dframe = data_dframe[,features]
    feature_matrix = data.matrix(feature_dframe)
    
    output_dframe = data_dframe[,output]
    output_matrix = data.matrix(output_dframe)
    return (list('feature_matrix'=feature_matrix,'output_matrix'=output_matrix))
}

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [3]:
example = get_matrix_data(train,'sqft_living','price')
example$feature_matrix[1,]
example$output_matrix[1]

constant sqft_living 
          1        1180

[1] 221900

# Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0\*1.0 + 1.0\*1180.0 = 1181.0 this is the dot product between these two arrays. If they're vectors we can use np.dot() to compute this:

In [4]:
#Predicting output given regression weights
my_weight = as.matrix(c(1,1))
my_feature = example$feature_matrix[1,]
predicted_value = sum(my_feature %*% my_weight)
predicted_value

[1] 1181

Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
#predict output
predict_output <- function(feature_matrix, weights){
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = feature_matrix %*% weights
    return(predictions)
}

In [6]:
test_predictions = predict_output(example$feature_matrix, my_weight)
print(test_predictions[1]) # should be 1181.0
print(test_predictions[2]) # should be 2571.0


[1] 1181
[1] 2571


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [7]:
# Computing the Derivative
feature_derivative <- function(err,feat){
    derivatives = 2*(err * feat)
    return(sum(derivatives))
}

In [8]:
example = get_matrix_data(sales, 'sqft_living','price')
my_weight = as.matrix(c(0,0))
test_predictions = predict_output(example$feature_matrix, my_weight)
errors = test_predictions - example$output_matrix
feature = example$feature_matrix[,1]
derivative = feature_derivative(errors,feature)
derivative
sum(-example$output_matrix)*2

[1] -23345850016

[1] -23345850016

# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [9]:
#GRADIENT DECENT
regression_gradient_decent <- function(feature_matrix, output, initial_weight,step_size, tolerance){
    converged = FALSE
    weights = initial_weight
    while(!(converged)){
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        error = predictions - output
        gradient_sum_square = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for(i in 1:length(weights)){
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative =  feature_derivative(error,feature_matrix[,i])
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_square = gradient_sum_square+(derivative**2)
            # subtract the step size times the derivative from the current weight
            weights[i] =weights[i]-(step_size*derivative)
        }
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_square)
        if(gradient_magnitude < tolerance){
            converged = TRUE
        } 
    }
    return(weights)
}

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

# Running the Gradient Descent as Simple Regression

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimat the parameters in the simple regression on squarefeet. The folowing cell sets up the feature_matrix, output, initial weights and step size for the first model:

In [10]:
#RUNNING GRADIENT DECENT
simple_feature <- 'sqft_living'
my_output <- 'price'
simple_matrix <- get_matrix_data(train,simple_feature,my_output)
initial_weight = c(-47000,1)
step_size=7e-12
tolerance = 2.5e7

Next run your gradient descent with the above parameters.

In [11]:
test_weight = regression_gradient_decent(simple_matrix$feature_matrix, simple_matrix$output_matrix,initial_weight,step_size,tolerance)
test_weight

[1] -46999.8872    281.9121

How do your weights compare to those achieved in week 1 (don't expect them to be exactly the same)? 

**Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [12]:
test_matrix <- get_matrix_data(test,'sqft_living','price')

Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [13]:
test_prediction <- predict_output(test_matrix$feature_matrix,test_weight)
test_prediction

356134.4
784640.9
435069.8
607036.2
260284.3
691609.9
1142669.3
587302.4
308209.4
728258.4
457622.8


**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?**

In [14]:
test_prediction[1]

[1] 356134.4

Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [15]:
test_residual <- test_prediction - test_matrix$output_matrix
test_RSS = sum(test_residual*test_residual)
test_RSS

[1] 2.754e+14

# Running a multiple regression

Now we will use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [16]:
feature <- c('sqft_living','sqft_living15')
my_output <- 'price'
matrix_test <- get_matrix_data(train,feature,my_output)
initial_weight <- as.matrix(c(-100000,1,1))
step_size <- 4e-12
tolerance <- 1e+9

Use the above parameters to estimate the model weights. Record these values for your quiz.

In [17]:
weight_2 <- regression_gradient_decent(matrix_test$feature_matrix, matrix_test$output_matrix,initial_weight,step_size,tolerance)
weight_2

-99999.96885
245.07260
65.27953


Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [18]:
matrix_test <- get_matrix_data(test,feature,my_output)

In [19]:
test_prediction_2 <- predict_output(matrix_test$feature_matrix,weight_2)
test_prediction_2

366651.4
762662.4
386312.1
636989.7
269618.0
713122.6
1091530.1
598292.3
293002.1
672521.8
465322.3


**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

In [20]:
test_prediction_2[1]

[1] 366651.4

What is the actual price for the 1st house in the test data set?

In [21]:
matrix_test$output_matrix[1]

[1] 310000

**Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?**

Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [22]:
test_residual <- test_prediction_2 - matrix_test$output_matrix
test_RSS = sum(test_residual*test_residual)
test_RSS

[1] 2.702634e+14

**Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data? **

Model 2 has the lowest RSS